<a href="https://colab.research.google.com/github/ruperty/colab/blob/master/learning_cartpole.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# install of my own pct library
!git clone https://github.com/ruperty/pct.git

Cloning into 'pct'...
remote: Enumerating objects: 366, done.
remote: Counting objects: 100% (366/366), done.
remote: Compressing objects: 100% (255/255), done.
remote: Total 366 (delta 217), reused 247 (delta 101), pack-reused 0
Receiving objects: 100% (366/366), 66.74 KiB | 7.42 MiB/s, done.
Resolving deltas: 100% (217/217), done.


In [2]:
import sys
import matplotlib.animation as animation
import plotly.graph_objects as go
from pct.dl.models.cartpole import CartpoleTuning

In [23]:
live_plot = sys.stdin.isatty()
offline_plot=True
training=True
render_gym=False
batch_size=10
num_runs=10000
prnt=True

In [24]:
# initial weights/gains
weights=[1.45,       0.4,       1.6,       -0.5]

In [32]:
ct = CartpoleTuning( "CartPole-v1")
ct.configure(weights=weights, opt_type="sgd", learning_rate=0.01, loss_type="rsuse",  loss_smooth=0.9, print=500, num_runs=num_runs)


TypeError: ignored

In [26]:
#plot = go.FigureWidget()
#plot.add_scatter(y=[]);
#plot.show()

In [27]:
ct.display_configure(live=False, render=render_gym)

In [28]:
out = ct.run( batch_size, training)        
#   ct.show()
#print(out)
        
#print("mean", ct.data.get_mean().numpy())
        


1000 loss 0.580      1.416      0.350      1.614      -0.500     
2000 loss 0.369      1.359      0.280      1.622      -0.500     
3000 loss 0.211      1.329      0.301      1.637      -0.500     
4000 loss 0.204      1.319      0.359      1.659      -0.500     
5000 loss 0.161      1.319      0.448      1.692      -0.500     
6000 loss 0.179      1.319      0.513      1.726      -0.500     
7000 loss 0.226      1.320      0.556      1.759      -0.500     
8000 loss 0.178      1.321      0.582      1.790      -0.500     
9000 loss 0.204      1.322      0.597      1.819      -0.500     
10000 loss 0.164      1.323      0.604      1.845      -0.500     


In [31]:
import plotly.graph_objects as go
edata=ct.get_error_data()  
fig = go.Figure()
fig.add_trace(go.Scatter(mode='lines',x=edata[0], y=edata[1], name="err"))
fig.update_layout(title="Global Error", xaxis_title="Time", yaxis_title="Error")
fig.show()

In [11]:
ct.close()       